<a href="https://colab.research.google.com/github/sjkim-audio/audio-dsp-lab/blob/main/notebooks/guitar_bass_separation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import files
from IPython.display import Audio, display

# 1. Demucs 라이브러리 설치함
# -q: 로그 출력 최소화
get_ipython().system('pip install -q demucs')

# 2. 오디오 파일 업로드함
print("파일을 업로드하세요 (.mp3, .wav 등)")
uploaded = files.upload()

if not uploaded:
    print("업로드된 파일이 없음")
else:
    filename = list(uploaded.keys())[0]

    # 3. 음원 분리 수행함
    # -n htdemucs_6s: 베이스와 기타가 포함된 6개 트랙 분리 모델 사용
    # --two-stems: 특정 악기만 빠르게 분리할 때 사용 가능하나, 정확도를 위해 전체 분리 후 선택
    print(f"[{filename}] 분석 및 분리 시작함...")
    get_ipython().system(f'demucs -n htdemucs_6s "{filename}"')

    # 4. 결과 경로 설정 및 확인함
    # Demucs 기본 출력 경로: separated/모델명/파일명/
    name_only = os.path.splitext(filename)[0]
    output_dir = os.path.join('separated', 'htdemucs_6s', name_only)

    bass_track = os.path.join(output_dir, 'bass.wav')
    guitar_track = os.path.join(output_dir, 'guitar.wav')

    # 5. 분리된 오디오 출력함
    if os.path.exists(output_dir):
        print("\n[분리 결과]")

        if os.path.exists(bass_track):
            print("Bass Track:")
            display(Audio(bass_track))

        if os.path.exists(guitar_track):
            print("Guitar Track:")
            display(Audio(guitar_track))

        print(f"전체 파일 경로: {output_dir}")
    else:
        print("분리 실패함. 파일 형식을 확인할 것")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 5.6 MB/s eta 0:00:00
파일을 업로드하세요 (.mp3, .wav 등)


Saving 무지성 원테이크 뭐시기.wav to 무지성 원테이크 뭐시기.wav
[무지성 원테이크 뭐시기.wav] 분석 및 분리 시작함...
Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/5c90dfd2-34c22ccb.th" to /root/.cache/torch/hub/checkpoints/5c90dfd2-34c22ccb.th
100% 52.4M/52.4M [00:00<00:00, 166MB/s]
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /content/separated/htdemucs_6s
Separating track 무지성 원테이크 뭐시기.wav
100%|██████████████████████████████████████████████████████████████████████| 216.45/216.45 [05:11<00:00,  1.44s/seconds]
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torchaudio/_torchcodec.py", line 246, in save_with_torchcodec
    from torchcodec.encoders import AudioEncoder
ModuleNotFoundError: No module named 'torchcodec'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/bin/demucs", line 8, in <module>
    sys.exit(main())
 